# Ranking investors based on historical data:

In this notebook the historically available data on what has happened to the companies (closed, went IPO, acquired or still operating) is used to generate a ranking for 44552  investors.  

In [1]:
from utils import *
from sklearn import preprocessing
%matplotlib inline

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



plotly  2.4.0


In [2]:
df1 = pd.read_csv('./data/companies_funding.csv')
df2 = pd.read_csv('./data/companies_status.csv')

In [3]:
df1.shape, df2.shape

((423918, 12), (690594, 6))

In [4]:
df_main = join_df(df1, df2, "ds_uuid", "ds_uuid", col_on = ['ds_uuid','status'])

In [5]:
df1.shape, df2.shape, df_main.shape

((423918, 12), (690594, 6), (423918, 13))

In [6]:
display(DataFrameSummary(df_main).summary())

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.py:111: DeprecationWarning:

pandas.core.common.is_numeric_dtype is deprecated. import from the public API: pandas.api.types.is_numeric_dtype instead

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pandas_summary/__init__.p

,Unnamed: 0,ds_uuid,investor,date,amount_raised,round_type,update_status,source,round_id,currency,clean_url,update_date,status
count,423918,NaN,NaN,NaN,278905,NaN,NaN,NaN,333596,NaN,NaN,NaN,NaN
mean,211958,NaN,NaN,NaN,3.44921e+07,NaN,NaN,NaN,2.17115,NaN,NaN,NaN,NaN
std,122375,NaN,NaN,NaN,1.57354e+09,NaN,NaN,NaN,1.61722,NaN,NaN,NaN,NaN
min,0,NaN,NaN,NaN,0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
25%,105979,NaN,NaN,NaN,1e+06,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
50%,211958,NaN,NaN,NaN,3.725e+06,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
75%,317938,NaN,NaN,NaN,1.3e+07,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN
max,423917,NaN,NaN,NaN,5.0005e+11,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN
counts,423918,423918,333504,333505,278905,333505,423606,333596,333596,284873,423918,423918,423798
uniques,423918,184146,44552,6428,15784,34,2,2,10,54,184146,201,4


In [7]:
df_main.status.value_counts()

operating    317539
acquired      60253
closed        24546
ipo           21460
Name: status, dtype: int64

The companies that are closed is consider to be a loss for their investors so we are going to mark them with a -4 score. On the other hand, the companies that managed to become IPO or acquired are reflecting the successes of the investor decision to invest in them. With this logic I mind I have labeled each company with a score that is a reflection of their operating status and afterword by summing these scores as the function of investor, each investor resaved a quality number.


In [8]:
def successes_value(x): 
    if x == 'operating':
        return(0)
    elif x == 'acquired':
        return (1)
    elif x == 'closed':
        return (-2)
    elif x == 'ipo':
        return (2)

In [9]:
def period(x): return (20)

In [10]:
df_main['exit_scores'] = df_main['status'].apply(successes_value)
df_main['period'] = df_main['status'].apply(period)

In [11]:
df_main.head()

,Unnamed: 0,ds_uuid,investor,date,amount_raised,round_type,update_status,source,round_id,currency,clean_url,update_date,status,exit_scores,period
0,0,c9005990-b1f3-4e27-a1fd-945bb8d16cfd,Menlo Ventures,2006-06-29,40000000.0,venture C,success,crunchbase,3.0,USD,epac.com,2018-01-21,operating,0.0,20
1,1,bc44e52d-c991-469d-b769-8571623437fa,NaN,NaN,NaN,NaN,no data,NaN,NaN,NaN,creditrepairexpert.org,2017-06-16,operating,0.0,20
2,2,90214116-40c1-472b-b0b4-3fc53fd8dc38,NaN,NaN,NaN,NaN,no data,NaN,NaN,NaN,xpertfinancial.com,2017-08-05,closed,-2.0,20
3,3,438ed619-7fdb-45ba-9788-5fce90536b46,NaN,NaN,NaN,NaN,no data,NaN,NaN,NaN,searchoptmedia.com,2017-07-06,operating,0.0,20
4,4,6ff5e99a-c45d-494b-b4cb-257b55d1bc73,NaN,NaN,NaN,NaN,no data,NaN,NaN,NaN,vision64.com,2017-06-16,operating,0.0,20


In [12]:
# df_g1 = df_main.groupby([ 'investor'],as_index = False)[['exit_scores']].sum()

# num_investments = df_main.groupby([ 'investor'],as_index = False)[['clean_url']].count()
# num_investments.head()

# result = pd.merge(df_g1, num_investments, on=['investor', 'investor'])
# result.head()

In [13]:
df_g1 = df_main.groupby([ 'investor' ],as_index = True)[['exit_scores']].sum()

## Lets look at the scores:

In [14]:
df_asc = df_g1.sort_values('exit_scores', ascending=True)
iplot(df_asc[:100].iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='quality score',
                 title='Worst investors'))

In [15]:
df_des = df_g1.sort_values('exit_scores', ascending=False)
iplot(df_des[1:1000].iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='quality score',
                 title='Best investors'), 
                )

In [16]:
plottest = df_des.reset_index()

data = [go.Bar(
            x=plottest.exit_scores[1:20].sort_values(),
            y=plottest.investor[1:20].tolist()[::-1],
            orientation = 'h'
)]

iplot(data)

In [17]:
df_des.loc['Index Ventures']

exit_scores    182.0
Name: Index Ventures, dtype: float64

## Top performing investors:


By exploring the data it becomes obvious that not all the 44552 are actively investing. For instance, there are many investment firms who has only invested couple of times throughout their history. Thus, to reduce the noise and also identify who are the main competitors to Index Ventures, we can start looking at the top 201 investor firms.

In this section, a second score is calculated by normalizing the “exit score” with the number of investments that each investment firm have made over the years. 


In [18]:
most = df_des[1:].copy()
most = most.dropna()

In [19]:
most.median(), most.mean()

(exit_scores    0.0
 dtype: float64, exit_scores    1.134587
 dtype: float64)

In [20]:
most.describe()

,exit_scores
count,44551.000000
mean,1.134587
std,8.486841
min,-69.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,497.000000


In [21]:
limited_df = most[most.exit_scores>=44]
limited_df.shape

(201, 1)

In [22]:
num_investments = df_main.groupby( ['investor'],as_index = True)['clean_url'].nunique()
num_investments.head()

investor
 A. James and Alice B. Clark Foundation    1
 ACME Investments                          1
 AERIUM Analytics                          1
 AI Life Sciences Investments              1
 Abhijeet  Birewar                         2
Name: clean_url, dtype: int64

In [23]:
result = pd.concat([limited_df ,num_investments], axis=1, join='inner')

In [24]:
result.shape, limited_df.shape, num_investments.shape

((201, 2), (201, 1), (44552,))

In [25]:
result.head()

,exit_scores,clean_url
investor,,
Sequoia Capital,497.0,731
New Enterprise Associates,430.0,695
Accel Partners,380.0,583
Intel Capital,355.0,718
Kleiner Perkins Caufield & Byers,313.0,441


In [26]:
result.loc['Sequoia Capital'] 

exit_scores    497.0
clean_url      731.0
Name: Sequoia Capital, dtype: float64

In [27]:
def norm (x,y):
    return (x/y)

In [28]:
result['norm_clean_url'] = np.vectorize(norm)(result['exit_scores'],result['clean_url'])

In [29]:
result= result.reset_index()
result.head()

,investor,exit_scores,clean_url,norm_clean_url
0,Sequoia Capital,497.0,731,0.679891
1,New Enterprise Associates,430.0,695,0.618705
2,Accel Partners,380.0,583,0.651801
3,Intel Capital,355.0,718,0.494429
4,Kleiner Perkins Caufield & Byers,313.0,441,0.709751


In [30]:
plot = result.groupby([ 'investor'],as_index = True)[['norm_clean_url']].sum()
df_des = plot.sort_values('norm_clean_url', ascending=False)

iplot(df_des[:200].iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='quality score',
                 title='Best investors'))

In [31]:
df_des

,norm_clean_url
investor,
Skyline Ventures,1.961538
Aisling Capital,1.902439
Boulder Ventures,1.853659
Clarus Ventures,1.818182
Brookside Capital,1.806452
HealthCap,1.781250
Domain Associates,1.705882
Alta Partners,1.691358
Employee Stock Option Fund (ESO Fund),1.657143


In [32]:
df_des.describe()

,norm_clean_url
count,201.000000
mean,0.842250
std,0.393281
min,0.036842
25%,0.570093
50%,0.784615
75%,1.084746
max,1.961538


In [33]:
df_des.loc['Sequoia Capital']

norm_clean_url    0.679891
Name: Sequoia Capital, dtype: float64

In [34]:
result[result.investor == 'Sequoia Capital']

,investor,exit_scores,clean_url,norm_clean_url
0,Sequoia Capital,497.0,731,0.679891


In [35]:
df_des.loc['Index Ventures']

norm_clean_url    0.524496
Name: Index Ventures, dtype: float64

In [36]:
result[result.investor == 'Index Ventures']

,investor,exit_scores,clean_url,norm_clean_url
16,Index Ventures,182.0,347,0.524496
